In [ ]:
from pyincore.analyses.montecarlofailureprobability import MonteCarloFailureProbability
from pyincore.analyses.buildingdamage import BuildingDamage
from pyincore.analyses.waterfacilitydamage import WaterFacilityDamage
from pyincore.analyses.bridgedamage import BridgeDamage
from pyincore.client import InsecureIncoreClient

### define client

In [ ]:
client = InsecureIncoreClient("http://incore2-services-dev.ncsa.illinois.edu:8888", "incrtest")

In [ ]:
mc = MonteCarloFailureProbability(client)
mc.load_remote_input_dataset("damage", "5a29782fc7d30d4af537ace5")
mc.set_parameter("result_name", "mc_failure_probability")
mc.set_parameter("num_cpu", 8)
mc.set_parameter("num_samples", 10)
mc.set_parameter("damage_interval_keys",
                 ["insignific", "moderate", "heavy", "complete"])
mc.set_parameter("failure_state_keys", ["moderate", "heavy", "complete"])
mc.run_analysis()

In [ ]:
mc.get_output_datasets()

### chaining with building damage

In [ ]:
bldg_dmg = BuildingDamage(client)
bldg_dmg.load_remote_input_dataset("buildings", "5a284f0bc7d30d13bc081a28")
bldg_dmg.load_remote_input_dataset("dmg_ratios", "5a284f2ec7d30d13bc08209a")
bldg_dmg.set_parameter("result_name", "memphis_building_damage")
bldg_dmg.set_parameter("mapping_id", "5b47b350337d4a3629076f2c")
bldg_dmg.set_parameter("hazard_type", "earthquake")
bldg_dmg.set_parameter("hazard_id", "5b902cb273c3371e1236b36b")
bldg_dmg.set_parameter("num_cpu", 1)
bldg_dmg.run_analysis()

In [ ]:
building_dmg_result = bldg_dmg.get_output_dataset('result')

mc = MonteCarloFailureProbability(client)
mc.set_input_dataset("damage", building_dmg_result)
mc.set_parameter("result_name", "bldg_mc_failure_probability")
mc.set_parameter("num_cpu", 8)
mc.set_parameter("num_samples", 10)
mc.set_parameter("damage_interval_keys",
                 ["insignific", "moderate", "heavy", "complete"])
mc.set_parameter("failure_state_keys", ["moderate", "heavy", "complete"])
mc.run_analysis()

### chaining with water facility damage

In [ ]:
wf_dmg = WaterFacilityDamage(client)
wf_dmg.load_remote_input_dataset("water_facilities", "5a284f2ac7d30d13bc081e52")
wf_dmg.set_parameter("result_name", "wf_dmg_results")
wf_dmg.set_parameter("hazard_type", "earthquake")
wf_dmg.set_parameter("hazard_id", "5b902cb273c3371e1236b36b")
wf_dmg.set_parameter("mapping_id", "5b47c383337d4a387669d592")
wf_dmg.set_parameter("fragility_key", "pga")
wf_dmg.set_parameter("use_liquefaction", True)
wf_dmg.set_parameter("liquefaction_geology_dataset_id", "5a284f53c7d30d13bc08249c")
wf_dmg.set_parameter("liquefaction_fragility_key", "pgd")
wf_dmg.set_parameter("use_hazard_uncertainty", False)
wf_dmg.set_parameter("num_cpu", 1)
wf_dmg.run_analysis()

In [ ]:
water_facility_dmg_result = wf_dmg.get_output_dataset('result')

mc = MonteCarloFailureProbability(client)
mc.set_input_dataset("damage", water_facility_dmg_result)
mc.set_parameter("result_name", "water_facility_mc_failure_probability")
mc.set_parameter("num_cpu", 8)
mc.set_parameter("num_samples", 10)
mc.set_parameter("damage_interval_keys",
                 ["none", "slight-mod", "mod-extens", "ext-comple", "complete"])
mc.set_parameter("failure_state_keys", ["slight-mod", "mod-extens", "ext-comple", "complete"])
mc.run_analysis()

### chaining with electric power facility damage

### chaining with bridge damage

In [ ]:
bridge_dmg = BridgeDamage(client)
bridge_dmg.load_remote_input_dataset("bridges", "5a284f2dc7d30d13bc082040")
bridge_dmg.load_remote_input_dataset("dmg_ratios", "5a284f2cc7d30d13bc081f96")
bridge_dmg.set_parameter("result_name", "bridge_dmg")
bridge_dmg.set_parameter("mapping_id", "5b47bcce337d4a37755e0cb2")
bridge_dmg.set_parameter("hazard_type", "earthquake")
bridge_dmg.set_parameter("hazard_id", "5b902cb273c3371e1236b36b")
bridge_dmg.set_parameter("num_cpu", 4)

bridge_dmg.run_analysis()

In [ ]:
bridge_dmg_result = bridge_dmg.get_output_dataset('result')

mc = MonteCarloFailureProbability(client)
mc.set_input_dataset("damage", bridge_dmg_result)
mc.set_parameter("result_name", "bridge_facility_mc_failure_probability")
mc.set_parameter("num_cpu", 8)
mc.set_parameter("num_samples", 10)
mc.set_parameter("damage_interval_keys",
                 ["none", "slight-mod", "mod-extens", "ext-comple", "complete"])
mc.set_parameter("failure_state_keys", ["slight-mod", "mod-extens", "ext-comple", "complete"])
mc.run_analysis()